In [38]:
import numpy as np
from tqdm import tqdm
import Bio.SeqIO as SeqIO

## Реализация алгоритмов Смита-Ватермана и Нидлмана-Вунша
Так как алгоритмы мало отличаются, я сделал для них обоих только одну функцию для вычисления матрицы скоров и одну - для обратного хода.

In [152]:
def calculate_matrix(x: str, y: str, 
                        scoring_matrix: np.ndarray[np.ndarray[int]], indel_cost: int,
                        algorithm='global') -> np.ndarray[np.ndarray[int]]:
    """
    Вычисляет матрицу скоров для алгоритмов Смита-Ватермана и Нидлмана-Вунша. Принимаются только ДНКовые последовательности.
    :param str x: Первая последовательность.
    :param str y: Вторая последовательность.
    :param np.ndarrat[np.ndarray[int]] scoring_matrix: Матрица 4x4, устанавливающая штрафы за разные мисметчи.
    :param int indel_cost: Штраф за инделы, отрицательное число.
    :param str algorithm: Алгоритм выравнивания: global - Нидлман-Вунш, local - Смит-Ватерман.
    :return: матрицу скоров, позицию максимума в виде кортежа (i, j) и матрицу обратного хода.
    """
    if len(y) > len(x): x, y = y, x
    x = list(x); y = list(y)
    nucl_to_idx = {'a':0, 't':1, 'g':2, 'c':3}
    
    score_matrix = np.zeros((len(x)+1, len(y)+1))
    if algorithm == 'global':
        score_matrix[0, ...] = np.arange(0, (len(y)+1)*indel_cost, indel_cost)
        score_matrix[..., 0] = np.arange(0, (len(x)+1)*indel_cost, indel_cost)
    elif algorithm == 'local':
        pass
    else:
        raise Exception('Invalid algorithm value, must be on of: needleman-wunsch, global, local')

    max_pos = (0, 0)
    backtrace_matrix = np.zeros((len(x)+1, len(y)+1))

    for i in tqdm(range(1, len(x)+1), desc='Calculating score matrix: '):
        for j in range(1, len(y)+1):
            cx = nucl_to_idx[x[i-1].lower()]; cy = nucl_to_idx[y[j-1].lower()]
            match_mismatch = score_matrix[i-1][j-1] + scoring_matrix[cx][cy]
            indel_x = score_matrix[i][j-1] + indel_cost
            indel_y = score_matrix[i-1][j] + indel_cost
            aux = np.array([match_mismatch, indel_x, indel_y])
            score_matrix[i][j] = aux.max()
            backtrace_matrix[i][j] = np.argmax(aux) + 1
                                     
            
            if score_matrix[i][j] > score_matrix[max_pos[0]][max_pos[1]]:
                max_pos = (i, j)

    return score_matrix, max_pos, backtrace_matrix

In [153]:
def backtrace_fun(x: str, y: str, backtrace_matrix: np.ndarray[np.ndarray[int]],
                  max_pos=None, score_matrix=None) -> tuple[str, str]:
    """
    Обратный ход выравнивания. Применим для обоих алгоритмов.
    :param x: Первая последовательность.
    :param y: Вторая последовательность.
    :param max_pos: Кортеж (i, j) - положение максимума.
    :param backtrace_matrix: Матрица обратного хода.
    :return: Две строки с выравниванием.
    
    """
    align1, align2 = '', ''
    if max_pos is not None:
        i, j = max_pos
    else:
        i, j = len(x), len(y)
    nucl_to_idx = {'a':0, 't':1, 'g':2, 'c':3}
    
    print('Backtracing...')
    while (i > 0 and j > 0) if max_pos is None else (score_matrix[i][j] != 0):
        if backtrace_matrix[i][j] == 1:
            align1 += x[i-1]
            align2 += y[j-1]
            i -= 1; j -= 1
        elif backtrace_matrix[i][j] == 3:
            align1 += x[i-1]
            align2 += '-'
            i -= 1
        elif backtrace_matrix[i][j] == 2:
            align1 += '-'
            align2 += y[j-1]
            j -= 1
    while i > 0:
        align1 += x[i-1]
        align2 += '-'
        i -= 1
    while j > 0:
        align1 += '-'
        align2 += y[j-1]
        j -= 1
    if max_pos is not None:
        if len(align2) > len(align1): align1, align2 = align2, align1
        align1 = '-'*(i-j) + x[:i] + align1 + x[max_pos[i]-1:] + '-'*(len(x) - j)

    return align1[::-1], align2[::-1]

In [154]:
x = 'AAAGTGA'
y = 'CAATGC'

s = np.array([[2, -1, -1, -1],
              [-1, 2, -1, -1],
              [-1, -1, 2, -1],
              [-1, -1, -1, 2]
])
d = -3

scores, max_pos, backtrace = calculate_matrix(x, y, s, d)
print('Scores:', *['\t'.join(map(str, row)) for row in scores], max_pos, sep='\n')
print()
print('Backtrace:', *['\t'.join(map(str, row)) for row in backtrace], max_pos, sep='\n')
print(*backtrace_fun(x, y, backtrace), sep='\n')

Calculating score matrix: 100%|██████████| 7/7 [00:00<00:00, 5944.55it/s]

Scores:
0.0	-3.0	-6.0	-9.0	-12.0	-15.0	-18.0
-3.0	-1.0	-1.0	-4.0	-7.0	-10.0	-13.0
-6.0	-4.0	1.0	1.0	-2.0	-5.0	-8.0
-9.0	-7.0	-2.0	3.0	0.0	-3.0	-6.0
-12.0	-10.0	-5.0	0.0	2.0	2.0	-1.0
-15.0	-13.0	-8.0	-3.0	2.0	1.0	1.0
-18.0	-16.0	-11.0	-6.0	-1.0	4.0	1.0
-21.0	-19.0	-14.0	-9.0	-4.0	1.0	3.0
(6, 5)

Backtrace:
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0	2.0	2.0	2.0
0.0	1.0	1.0	1.0	2.0	2.0	2.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	3.0	3.0	1.0	1.0	2.0
0.0	1.0	3.0	3.0	1.0	1.0	1.0
0.0	1.0	3.0	3.0	3.0	1.0	2.0
0.0	1.0	1.0	1.0	3.0	3.0	1.0
(6, 5)
Backtracing...
AAAGTGA
CAA-TGC


Это выравнивание первых 100 нуклеотидов последовательностей инсулина человека и морской свинки.

In [155]:
x = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[0:100]
y = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[0:100]

scores, max_pos, backtrace = needleman_wunsch_matrix(x, y, s, d)
#print('Scores:', *['\t'.join(map(str, row)) for row in scores], max_pos, sep='\n')
#print()
#print('Backtrace:', *['\t'.join(map(str, row)) for row in backtrace], max_pos, sep='\n')
print(*backtrace_fun(x, y, backtrace), sep='\n')

Calculating score matrix: 100%|██████████| 100/100 [00:00<00:00, 933.17it/s]

Backtracing...
CT-C-GA-GGGGC-CTA-GACATTGCCCTCCAGAGAGAGCACCCAACACCCTCCAGGCTTGACCGGCCAGGGTGTC-CCCTTCCTACCT-TGGAGAGAGCAGCCCCA
CTGCAGACCCAGCACCAGGGAAATG--ATCCAGAAATTGCAACC-TCAGCCCCCTGGC--CATCTG-C-TGATGCCACCACCCCCAGGTCCCTAATGGGCCTGGTGG


In [156]:
x = 'AAAGTGA'
y = 'CAATGC'

s = np.array([[2, -1, -1, -1],
              [-1, 2, -1, -1],
              [-1, -1, 2, -1],
              [-1, -1, -1, 2]
])
d = -3

scores, max_pos, backtrace = calculate_matrix(x, y, s, d, algorithm='local')
print('Scores:', *['\t'.join(map(str, row)) for row in scores], max_pos, sep='\n')
print()
print('Backtrace:', *['\t'.join(map(str, row)) for row in backtrace], max_pos, sep='\n')
print(*backtrace_fun(x, y, backtrace, max_pos, scores), sep='\n')

Calculating score matrix: 100%|██████████| 7/7 [00:00<00:00, 7602.31it/s]

Scores:
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	-1.0	2.0	2.0	-1.0	-1.0	-1.0
0.0	-1.0	1.0	4.0	1.0	-2.0	-2.0
0.0	-1.0	1.0	3.0	3.0	0.0	-3.0
0.0	-1.0	-2.0	0.0	2.0	5.0	2.0
0.0	-1.0	-2.0	-3.0	2.0	2.0	4.0
0.0	-1.0	-2.0	-3.0	-1.0	4.0	1.0
0.0	-1.0	1.0	0.0	-3.0	1.0	3.0
(4, 5)

Backtrace:
0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	1.0	1.0	1.0
0.0	1.0	1.0	1.0	1.0	1.0	2.0
0.0	1.0	1.0	1.0	1.0	3.0	1.0
0.0	1.0	1.0	1.0	3.0	1.0	1.0
0.0	1.0	1.0	1.0	2.0	3.0	1.0
(4, 5)
Backtracing...
-------AGTG-AAAG
CAATG


In [157]:
x = list(SeqIO.parse('human.fasta', 'fasta'))[0].seq[0:10]
y = list(SeqIO.parse('guinea_pig.fasta', 'fasta'))[0].seq[0:30]

scores, max_pos, backtrace = calculate_matrix(x, y, s, d, algorithm='local')
print(*backtrace_fun(x, y, backtrace, max_pos, scores), sep='\n')

Calculating score matrix: 100%|██████████| 30/30 [00:00<00:00, 5844.36it/s]

Backtracing...


IndexError: string index out of range